# Agent-based model - Modules and Classes*
### *This code belongs to the paper "Assessing School-based Policy Actions for COVID-19: An Agent-Based Analysis of Incremental Infection Risk - CBM 2021"

In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import time
import pandas as pd
import numpy as np
import pyspark.sql.functions as f
from pyspark.sql.window import Window
import socket    
import random
from scipy import integrate
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import json 
import scipy as sc

hostname = socket.gethostname()    
IPAddr = socket.gethostbyname(hostname)  

conf = SparkConf().setAll([("spark.executor.instances", '5'), ('spark.executor.memory', '8g'), ('spark.executor.cores', '5'), ('spark.driver.memory','2g'),('spark.sql.broadcastTimeout', '3000')])
conf.setMaster('yarn')
conf.setAppName('spark-yarn-2')
conf.set("spark.driver.host", IPAddr)

pd.options.display.max_rows = 100

In [2]:
class Agent:
    
    
    
    def __init__ (self, agent_id = 0, gender = 0, stress_level = 0, resilience = 0, HEALTH_STATUS = 0, test_result = 'not_tested', class_presence = 0, prevalence_rate = 0, n = 1, time = 0):
        self.agent_id = agent_id
        self.gender = gender

        if (self.gender == 'Female')| (self.gender == 'female'):
            self.adherence_to_rules = 0.88
        elif (self.gender == 'Male') | (self.gender == 'male'):
            self.adherence_to_rules = 0.83
            
        if (stress_level == 'High') | (stress_level == 'high'):
            self.stress_level_impact = 0.6
        else:
            self.stress_level_impact = 1
            
        if (resilience == 'High') | (resilience == 'high'): #reduces 46% of stress
            self.resilience_impact = 0.54
        else:
            self.resilience_impact = 1    

        self.HEALTH_STATUS = HEALTH_STATUS
        if (self.HEALTH_STATUS == 'Asymptomatic') | (self.HEALTH_STATUS == 'asymptomatic') | (self.HEALTH_STATUS == 'Pre_symptomatic') | (self.HEALTH_STATUS == 'pre_symptomatic') | (self.HEALTH_STATUS == 'Mildly_symptomatic') | (self.HEALTH_STATUS == 'mildly_symptomatic') | (self.HEALTH_STATUS == 'Symptomatic') | (self.HEALTH_STATUS == 'symptomatic'):
            self.infection_status = 1
        else:
            self.infection_status = 0 #if exposed, infection_status will still be 0.
        self.test_result = test_result
        self.class_presence = class_presence
        self.prevalence_rate = prevalence_rate
        self.n = n
        self.incubation_period = np.random.normal(5.75, 5.75/3)
        self.latent_period = np.random.normal(2, 2/3)
        self.recovery_period = 14
        self.time = time
        self.log = pd.DataFrame({'time' : [], 'HEALTH_STATUS' : [], 'test_result' : []})
        
        
#-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------        
        
    def __repr__(self):
        return ('{}, {}, {}, {}, {}, {}'.format(self.agent_id, self.gender, self.stress_level_impact, self.resilience_impact, self.HEALTH_STATUS, self.class_presence))

    
#-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
              
        
    def add_agent_health_log(self, time):
        self.time = time
        logs = self.log
        
        if self.HEALTH_STATUS == 'Exposed':
            logs = logs[logs.HEALTH_STATUS == 'Exposed']
            if len(list(logs.time)) > 0:
                delta_time = list(logs.time)[-1] - list(logs.time)[0]
                if delta_time <= self.latent_period:
                    self.HEALTH_STATUS = 'Exposed'
                    self.infection_status = 0
                elif delta_time > self.latent_period:
                    self.HEALTH_STATUS = random.choices(['Asymptomatic','Pre_symptomatic'], k = 1, weights = [0.40,0.60])[0]  
                    self.infection_status = 1
                    
                    
        elif self.HEALTH_STATUS == 'Pre_symptomatic':
            logs = logs[logs.HEALTH_STATUS == 'Pre_symptomatic']
            if len(list(logs.time)) > 0:
                delta_time = list(logs.time)[-1] - list(logs.time)[0]
                if delta_time <= self.incubation_period - self.latent_period :
                    self.HEALTH_STATUS = 'Pre_symptomatic'
                    self.infection_status = 1
                elif delta_time > self.incubation_period - self.latent_period:
                    self.HEALTH_STATUS = random.choices(['Mildly_symptomatic', 'Symptomatic'], k = 1, weights = [0.81, 0.19])[0]  
                    self.infection_status = 1
        
        elif (self.HEALTH_STATUS == 'Mildly_symptomatic'):
            logs = logs[(logs.HEALTH_STATUS == 'Mildly_symptomatic')]
            if len(list(logs.time)) > 0:
                delta_time = list(logs.time)[-1] - list(logs.time)[0]
                if delta_time <= self.recovery_period :
                    self.HEALTH_STATUS = 'Mildly_symptomatic'
                    self.infection_status = 1
                elif delta_time > self.recovery_period:
                    self.HEALTH_STATUS = 'Immune'
                    self.infection_status = 0
                    
        elif (self.HEALTH_STATUS == 'Symptomatic'):
            logs = logs[(logs.HEALTH_STATUS == 'Symptomatic')]
            if len(list(logs.time)) > 0:
                delta_time = list(logs.time)[-1] - list(logs.time)[0]
                if delta_time <= self.recovery_period :
                    self.HEALTH_STATUS = 'Symptomatic'
                    self.infection_status = 1
                elif delta_time > self.recovery_period:
                    self.HEALTH_STATUS = random.choices(['Immune', 'Deceased'], k = 1, weights = [0.977, 0.023])[0]
                    self.infection_status = 0
                    
        elif self.HEALTH_STATUS == 'Asymptomatic':
            logs = logs[logs.HEALTH_STATUS == 'Asymptomatic']
            if len(list(logs.time)) > 0:
                delta_time = list(logs.time)[-1] - list(logs.time)[0]
                if delta_time <= self.recovery_period:
                    self.HEALTH_STATUS = 'Asymptomatic'
                    self.infection_status = 1
                elif delta_time > self.recovery_period:
                    self.HEALTH_STATUS = 'Immune'
                    self.infection_status = 0
                    
        new_row = {'time' : time, 'HEALTH_STATUS' : self.HEALTH_STATUS, 'test_result' : self.test_result}
        self.log = self.log.append(new_row, ignore_index=True)  
        

#-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    
    
    def update_agent_test_log(self, time, test_results_delay, test_accuracy): #if test_schedule _date == (current) time:
        self.time = time
        logs = self.log
        
        if self.test_result == 'Tested':
            logs = logs[logs.test_result == 'Tested']
            if len(list(logs.time)) > 0:
                delta_thing = len(list(logs.time))
                
                if np.mod(delta_thing, test_results_delay) != 0: #Still less than 2 days has passed from the test date
                    self.test_result = 'Tested'
                    
                elif np.mod(delta_thing, test_results_delay) == 0: #now let's ACTUALLY perform the test
                    u_prime = random.random()
                    if u_prime <= test_accuracy: 
                        if self.infection_status == 1:
                            self.test_result = 'Positive'
                        else:
                            self.test_result = 'Negative'
                    else:
                        self.test_result = 'Negative'
                        
            
        self.log.loc[self.log.time == time, 'test_result'] = self.test_result
    
    
#-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
        
        
    
    def make_decision(self):
        if (self.HEALTH_STATUS == 'Asymptomatic') | (self.HEALTH_STATUS == 'asymptomatic') | (self.HEALTH_STATUS == 'Healthy') | (self.HEALTH_STATUS == 'healthy') | (self.HEALTH_STATUS == 'Pre_symptomatic') | (self.HEALTH_STATUS == 'pre_symptomatic') | (self.HEALTH_STATUS == 'Immune') | (self.HEALTH_STATUS == 'Exposed') :
            probability_of_showing_up = 1            
        elif (self.HEALTH_STATUS == 'Mildly_symptomatic') | (self.HEALTH_STATUS == 'mildly_symptomatic'):
            if (self.stress_level_impact == 0.6):
                probability_of_showing_up = (1 - self.adherence_to_rules) * (1 - self.stress_level_impact * self.resilience_impact)
            elif (self.stress_level_impact == 1):
                probability_of_showing_up = (1 - self.adherence_to_rules)
                
        elif (self.HEALTH_STATUS == 'Symptomatic') | (self.HEALTH_STATUS == 'symptomatic') | (self.HEALTH_STATUS == 'Deceased'):
             probability_of_showing_up = -1
        
        if (self.time == 0) & (self.agent_id == 1000 + self.n): #patient zero
            probability_of_showing_up = 1
                
        return(probability_of_showing_up) # 2%, 3% or 5%


#-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    
    
    def create_students(self, prevalence_rate, n):
        students = []
        for i in range(n-1):
            student_id = 1000 + i
            gender = random.choices(['Male', 'Female'], k = 1, weights = [0.58, 0.42])[0]
            stress_level = random.choices(['high', 'low'], k = 1, weights = [0.5, 0.5])[0]
            resilience = random.choices(['high', 'low'], k = 1, weights = [0.5, 0.5])[0]
            #we assume that in the begining, half of the pre-symptomatic have already developed symptoms and half of them stil do not.
            HEALTH_STATUS = random.choices(['Healthy', 'Asymptomatic', 'Pre_symptomatic' , 'Mildly_symptomatic', 'Symptomatic'], k = 1, weights = [(1 - prevalence_rate), prevalence_rate*0.40, prevalence_rate*0.6*0.5, prevalence_rate*0.6*0.5*0.81, prevalence_rate*0.6*0.5*0.19])[0]           
            class_presence = -2
            if HEALTH_STATUS == 'Symptomatic': 
                test_result = 'Positive'
            else:
                test_result = 'Not_tested'                
            time = 0
            student = Agent(student_id, gender, stress_level, resilience, HEALTH_STATUS, test_result, class_presence, prevalence_rate, n, time)
            student.add_agent_health_log(time)
            students.append(student)
        
        #patient zero: the initial infectious agent
        student_id = 1000 + n
        gender = random.choices(['Male', 'Female'], k = 1, weights = [0.58, 0.42])[0]
        stress_level = random.choices(['high', 'low'], k = 1, weights = [0.5, 0.5])[0]
        resilience = random.choices(['high', 'low'], k = 1, weights = [0.5, 0.5])[0]
        HEALTH_STATUS = random.choices(['Asymptomatic', 'Pre_symptomatic' ], k = 1, weights = [0.40, 0.60])[0]           
        class_presence = -2
        time = 0
        
        the_initial_sick_student = Agent(student_id, gender, stress_level, resilience, HEALTH_STATUS, test_result, class_presence, prevalence_rate, n, time)
        the_initial_sick_student.add_agent_health_log(time)
        students.append(the_initial_sick_student)
        
        return(students)   
    

#-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------


    def create_teacher(self):
        teacher_id = 2
        gender = random.choices(['Male', 'Female'], k = 1, weights = [0.50, 0.50])[0]
        stress_level = random.choices(['high', 'low'], k = 1, weights = [0.5, 0.5])[0]
        resilience = random.choices(['high', 'low'], k = 1, weights = [0.5, 0.5])[0]
        HEALTH_STATUS = 'Healthy'
        class_presence = 2
        test_result = 'Not_tested'
        prevalence_rate = 0
        n = 1
        time = 0
        teacher = Agent(teacher_id, gender, stress_level, resilience, HEALTH_STATUS, test_result, class_presence, prevalence_rate, n, time)
        teacher.add_agent_health_log(time)
        return(teacher)

In [3]:
class Classroom:
    
    def __init__ (self, students, teacher, rows, columns, prevalence_rate):
        self.students = students
        self.teacher = teacher
        self.n = len(students) 
        self.rows = rows
        self.columns = columns
        self.prevalence_rate = prevalence_rate
        
    def __repr__(self):
        return ('The class of "Classroom" returns the position matrix of classroom and students.')
    
    
    def get_classroom(self): 
        n = self.n
        rows = self.rows
        columns = self.columns
        students = self.students
        teacher = self.teacher
        teacher_positions = []
        for i in range(columns):
            teacher_positions.append(teacher)
        classroom_linear = []
        
        for student in students: #who will show up? who won't? (all healthy studetns will show up, though asymptomatics or Mildly_symptomatic should make a decision)
            u = random.random()
            if u <= student.make_decision():
                classroom_linear.append(student)
                student.class_presence = student.infection_status 
            else:
                student.class_presence = -1 
                
            if student.HEALTH_STATUS == 'Deceased':
                student.class_presence = -1 

            if student.test_result == 'Positive': # if test result are positive (false positive or true positive), they will NOT show up!
                student.class_presence = -1
                
            if (student.test_result == 'Tested') & (student.log.loc[student.log.test_result != 'Tested', 'test_result'].values[-1] == 'Positive'): 
                student.class_presence = -1
            
                
        n = len(classroom_linear)
        np.random.shuffle(classroom_linear)
        classroom_grid = np.zeros((rows+1, columns))
        classroom_linear = np.array(classroom_linear)
        if n > rows * columns:
            return(['ERROR: Number of students more than class capacity', 'ERROR: Number of students more than class capacity'])
        
        elif n == rows * columns:
            classroom_grid = classroom_linear.reshape((rows, columns))
            classroom_grid = np.vstack([teacher_positions, classroom_grid])

        elif n < rows * columns:
            empty_seats = []
            for i in range(rows * columns - n):
                empty_seats.append(Agent(-1, -1, -1, class_presence = -1))
            classroom_linear = np.append(classroom_linear, empty_seats)
            np.random.shuffle(classroom_linear)
            classroom_grid = classroom_linear.reshape((rows, columns))
            classroom_grid = np.vstack([teacher_positions, classroom_grid])
        
        classroom_scheme = np.zeros((rows + 1, columns))
        i = 0
        for row in classroom_grid:
            j = 0
            for column in row:
                if classroom_grid[i,j].class_presence == 1:
                    if classroom_grid[i,j].infection_status == 1:
                        classroom_scheme[i,j] = 1
                    else:
                        classroom_scheme[i,j] = 0
                elif classroom_grid[i,j].class_presence == -1:
                    classroom_scheme[i,j] = -1
                elif classroom_grid[i,j].class_presence == 2:
                    classroom_scheme[i,j] = 2
                j = j + 1
            i = i + 1
        return([classroom_grid, classroom_scheme])


In [4]:
class Quanta_Cone:
    
    def __init__ (self, d, r, h, concentration_rate, itter_count):
        self.d = d
        self.r = r
        self.h = h
        self.concentration_rate = concentration_rate
        self.itter_count = itter_count
        
    def __repr__(self):
        return ('The class of "Quanta_Cone" returns the particle distribution of quanta made by an infected person')
    
    def get_particle_ditribution(self):
        r = self.r
        h = self.h
        itter_count = self.itter_count
        particle_ditribution = []
        
        n_list = [1]
        for i in range(int(h*10)):
            n_list.append((((i+1)**2)/((i+2)**2)))

        n_list_new = [itter_count]
        for i in range(len(n_list)):
            j = n_list_new[i]*n_list[i]
            n_list_new.append(np.int(np.round(j)))

        n_list_new = n_list_new[1:-1]
        
        k = 0
        for j in n_list_new:
            k = k + 1
            z = 0.1*k
            for i in range(j):
                x = random.uniform(-r*z/h , r*z/h)
                y = random.uniform(-np.sqrt(np.power(r*z/h,2) - np.power(x,2)), np.sqrt(np.power(r*z/h,2) - np.power(x,2)))
                particle_ditribution.append([x, y, z])

        return(particle_ditribution)
    
    def get_individual_distribution_matrix(self):
        r = self.r
        h = self.h
        d = self.d
        k_z = int(np.ceil((h-d/2)/d)+1)
        k_x = int((2*np.ceil((r-d/2)/d)) + 1)
        distribution_matrix = np.zeros((k_z, k_x))
        
        column = []
        i = 1
        for ross in distribution_matrix:  
            row = []
            for j in range(1, len(list(ross))+1):
                element = [[((-(k_x/2) + j - 1)*d), ((-(k_x/2) + j)*d)], [((k_z - i-1)*d + d/2), ((k_z - i)*d + d/2)]]
                row.append(element)
            column.append(row)
            i = i + 1
            
        distribution_matrix = np.array(column)
        return(distribution_matrix)
    
    def get_individual_quanta_proportion(self):
        particle_ditribution = self.get_particle_ditribution()
        distribution_matrix = self.get_individual_distribution_matrix()
        quanta_proportion = np.zeros((int(distribution_matrix.shape[0]), int(distribution_matrix.shape[1])))
        
        i = 0
        for row in distribution_matrix:
            j = 0
            for mini_matrix in row:
                x_range = mini_matrix[0]
                z_range = mini_matrix[1]
                count = 0
                for point in particle_ditribution:
                    if (x_range[0] <= point[0]) & (point[0] <= x_range[1]) & (z_range[0] <= point[2]) & (point[2] <= z_range[1]):
                        count = count + 1
                        
                quanta_proportion[i,j] = (count/itter_count)
                
                j = j +1
            i = i + 1  
        return (quanta_proportion)

In [5]:
class Covid19_Spread:
    
    def __init__ (self, teacher, students, classroom_grid, quanta_proportion, d, ERq, IVRR, IR, n0, T, classroom_height, final_duration):
        self.teacher = teacher
        self.students = students
        self.classroom_grid = classroom_grid
        self.quanta_proportion = quanta_proportion
        self.d = d
        self.ERq = ERq
        self.IVRR = IVRR
        self.IR = IR
        self.n0 = n0
        self.T = T
        self.classroom_height = classroom_height
        self.final_duration = final_duration
        
    def __repr__(self):
        return ('The class of "Covid19_Spread" returns the matrix of infection rist for each studnet in the classroom')
    
    
    def get_total_quanta_matrix(self): #quanta DIRECTLY caused by infected students
        classroom_grid = self.classroom_grid
        quanta_proportion = self.quanta_proportion 
        ERq = self.ERq
        direct_quanta_matrix = np.zeros((int(classroom_grid.shape[0]), int(classroom_grid.shape[1])))
        infected_location = int(np.floor(quanta_proportion.shape[1]/2)) #where the infected person is lovated in the distribution matrix (column-var ;))        
        quanta_proportion[-1,infected_location] = 1000
        classroom_rows = classroom_grid.shape[0]
        classroom_columns = classroom_grid.shape[1]
        number_of_infected_s = 0
        
        for i in range(classroom_rows):
            for j in range(classroom_columns):
                if (classroom_grid[i, j].infection_status == 1) & (classroom_grid[i, j].class_presence == 1): # an infected STUDENT is in cell(i,j) of the classroom - it should NOT BE A TEACHER
                    number_of_infected_s = number_of_infected_s + 1
                    effective_area = classroom_grid[:i + 1, :] #رو به روی اون استودنت
                    
                    if  effective_area.shape[0] < quanta_proportion.shape[0]: #at the front of the student there is a wall so some of the particles will reflect
                        new_quanta_proportion = quanta_proportion[int(quanta_proportion.shape[0] - effective_area.shape[0]):, :]     
                    else:
                        new_quanta_proportion = quanta_proportion
                    if (j < infected_location) & ((classroom_columns-1) - j >= infected_location): #in the left side of the student there is a wall so some of the particles will reflect
                        new_quanta_proportion = new_quanta_proportion[:, (infected_location - j):]               
                    elif ((classroom_columns-1) - j < infected_location) & (j >= infected_location): #in the right side of the student there is a wall so some of the particles will reflect
                        new_quanta_proportion = new_quanta_proportion[:, : ((classroom_columns-1) - j) + infected_location + 1]
                    elif ((classroom_columns-1) - j < infected_location) & (j < infected_location): #Both walls
                        new_quanta_proportion = new_quanta_proportion[:, (infected_location - j):((classroom_columns-1) - j) + infected_location + 1]
                    else:
                        new_quanta_proportion = new_quanta_proportion
                        
                    infected_j = int(np.where(new_quanta_proportion == 1000)[1][0])
                    direct_quanta_matrix[(i - new_quanta_proportion.shape[0] + 1):i + 1, (j - infected_j):(j - infected_j) + int(new_quanta_proportion.shape[1])] = direct_quanta_matrix[(i - new_quanta_proportion.shape[0] + 1):i + 1, (j - infected_j):(j - infected_j) + int(new_quanta_proportion.shape[1])] + new_quanta_proportion
        
        direct_quanta_matrix = direct_quanta_matrix * ERq
        indirect_quanta_matrix = (1/(classroom_grid.shape[0]*classroom_grid.shape[1]))*np.ones((int(classroom_grid.shape[0]), int(classroom_grid.shape[1]))) * (number_of_infected_s * ERq)
        total_quanta_matrix = direct_quanta_matrix + indirect_quanta_matrix
        total_quanta_matrix[0, :] = sum(total_quanta_matrix[0, :]) #summation of all quanta for the teacher
        k = 0
        for row in classroom_grid:
            for p in (range(len(list(row)))):
                if (classroom_grid[k,p].infection_status == 1) | (classroom_grid[k,p].class_presence == -1) :
                    total_quanta_matrix[k,p] = 0.0  
            k = k + 1
        return([total_quanta_matrix, direct_quanta_matrix, indirect_quanta_matrix]) 
    
    
    def get_Wells_Riley_cumulative_risk(self): #quanta DIRECTLY caused by infected students
        classroom_grid = self.classroom_grid
        quanta_proportion = self.quanta_proportion 
        d = self.d
        ERq = self.ERq
        IVRR =  self.IVRR
        IR = self.IR
        n0 = self.n0
        T = self.T 
        classroom_height = self.classroom_height
        total_quanta_matrix = self.get_total_quanta_matrix()[0]
        t_range = np.arange(0, T+1, 1)     
        
        Wells_Riley_cumulative_risk = []
        Cu_Risk = []
        
        for ERq_I in total_quanta_matrix[0, :]:
            results = []
            V = int(classroom_grid.shape[1])*d*d*classroom_height
            def f(t):
                R = (ERq_I)/(IVRR*V) + (n0 + ERq_I/IVRR)*np.exp(-IVRR*t)/V
                return R
            for ttt in list(t_range):
                results.append(1 - np.exp(-IR*(integrate.quad(f,0,ttt/60)[0])))
            Cu_Risk.append(results)
        Wells_Riley_cumulative_risk.append(Cu_Risk)
        
        for row in total_quanta_matrix[1:, :]:
            Cu_Risk = []
            for ERq_I in row:
                results = []
                V = d*d*classroom_height
                def f(t):
                    R = (ERq_I)/(IVRR*V) + (n0 + ERq_I/IVRR)*np.exp(-IVRR*t)/V
                    return R
                for ttt in list(t_range):
                    results.append(1 - np.exp(-IR*(integrate.quad(f,0,ttt/60)[0])))
                Cu_Risk.append(results)
            Wells_Riley_cumulative_risk.append(Cu_Risk)
        Wells_Riley_cumulative_risk = np.array(Wells_Riley_cumulative_risk)    
        return(Wells_Riley_cumulative_risk)
    
    
    def get_final_Wells_Riley_risk(self):
        classroom_grid = self.classroom_grid
        quanta_proportion = self.quanta_proportion 
        d = self.d
        ERq = self.ERq
        IVRR =  self.IVRR
        IR = self.IR
        n0 = self.n0
        final_duration = self.final_duration 
        classroom_height = self.classroom_height
        total_quanta_matrix = self.get_total_quanta_matrix()[0]
        
        final_Wells_Riley_risk = []
        Cu_Risk = []
        
        for ERq_I in total_quanta_matrix[0, :]:
            V = int(classroom_grid.shape[1])*d*d*classroom_height
            def f(t):
                R = (ERq_I)/(IVRR*V) + (n0 + ERq_I/IVRR)*np.exp(-IVRR*t)/V
                return R
            Cu_Risk.append(1 - np.exp(-IR*(integrate.quad(f,0,final_duration/60)[0])))
        final_Wells_Riley_risk.append(Cu_Risk)
        
        for row in total_quanta_matrix[1:, :]:
            Cu_Risk = []
            for ERq_I in row:
                V = d*d*classroom_height
                def f(t):
                    R = (ERq_I)/(IVRR*V) + (n0 + ERq_I/IVRR)*np.exp(-IVRR*t)/V
                    return R
                Cu_Risk.append(1 - np.exp(-IR*(integrate.quad(f,0,final_duration/60)[0])))
            final_Wells_Riley_risk.append(Cu_Risk)
        final_Wells_Riley_risk = np.array(final_Wells_Riley_risk)    
        return(final_Wells_Riley_risk)
    
    

In [6]:
class Simulation:
    
    def __init__ (self, students, teacher, classroom_grid, covid19_spread_model, time):
        self.covid19_spread_model = covid19_spread_model
        self.classroom_grid = classroom_grid
        self.time = time
        self.students = students
        self.teacher = teacher

    
    def simulate_one_session_of_the_class(self):
        final_Wells_Riley_risk = self.covid19_spread_model.get_final_Wells_Riley_risk()
        classroom_grid = self.covid19_spread_model.classroom_grid
        students = self.students
        teacher = self.teacher
        
        axis_0 = 0
        for row in final_Wells_Riley_risk:
            axis_1 = 0
            for risk in row:
                if classroom_grid[axis_0, axis_1].HEALTH_STATUS == 'Healthy': #make sure not to update health status for already infected agents
                    q_id = classroom_grid[axis_0, axis_1].agent_id
                    if teacher.agent_id == q_id:
                        teacher.HEALTH_STATUS = random.choices(['Healthy', 'Exposed'], k = 1, weights = [(1-risk), risk])[0]                                                                                
                    for i in range(len(students)):
                        if students[i].agent_id == q_id:
                            students[i].HEALTH_STATUS = random.choices(['Healthy', 'Exposed'], k = 1, weights = [(1-risk), risk])[0] 
                    
                axis_1 = axis_1 + 1
            axis_0 = axis_0 + 1
        teacher.time = time
        teacher.add_agent_health_log(time) 
        for s in students:
            s.time = time
            s.add_agent_health_log(time)
            
        return([teacher, students])       
    
    
#!!!!! ISSUE: The first test is ready in 24 hrs! the rest in 48 hrs!   
    def perform_surveillance_testing(self, time, sample_percentage = 0.1, test_accuracy = 0.90, test_results_delay = 2):
        
        classroom_grid = self.covid19_spread_model.classroom_grid
        students = self.students
        teacher = self.teacher
        
        #pick sample_percentage of the class
        for student in students: #A student might not be present in the class, yet be picked for a test
            if (student.test_result == 'Not_tested') | (student.test_result == 'Negative'):
                u = random.random()
                if u <= sample_percentage: #chosen to be tested
                    student.test_result = 'Tested'
            
            if (student.HEALTH_STATUS == 'Symptomatic') | (student.test_result == 'Positive'):
                student.test_result = 'Tested'
                
            student.update_agent_test_log(time, test_results_delay, test_accuracy) #OVERRIDE
        return([teacher, students])
            
        
    
    def perform_contact_tracing(self, time, contact_tracing_level = 0, test_accuracy = 0.9, test_results_delay = 2):
        
        classroom_grid = self.covid19_spread_model.classroom_grid
        students = self.students
        teacher = self.teacher                                    


        if (contact_tracing_level > 0):
            for student in students: #No need to test students who are already tested positive
                if (student.test_result == 'Not_tested') | (student.test_result == 'Negative'):
                    u = random.random()
                    if u <= contact_tracing_level: #chosen to be tested
                        student.test_result = 'Tested'

                student.update_agent_test_log(time, test_results_delay, test_accuracy) #OVERRIDE  
        
        return([teacher, students])
    
    
    
    def get_classroom_summary(self, time):
        students = self.students
        teacher = self.teacher
        
        number_of_infected_students = 0
        
        number_of_healthy_students = 0
        number_of_exposed_students = 0
        number_of_asymptomatic_students = 0
        number_of_presymptomatic_students = 0
        number_of_mildly_symptomatic_students = 0
        number_of_symptomatic_students = 0
        number_of_immune_students = 0
        number_of_deseased_students = 0
        
        teacher_status = teacher.HEALTH_STATUS
        
        for student in students:
            if student.HEALTH_STATUS == 'Healthy':
                number_of_healthy_students = number_of_healthy_students + 1
            elif student.HEALTH_STATUS == 'Exposed':
                number_of_exposed_students = number_of_exposed_students + 1
            elif student.HEALTH_STATUS == 'Asymptomatic':
                number_of_asymptomatic_students = number_of_asymptomatic_students + 1
            elif student.HEALTH_STATUS == 'Pre_symptomatic':
                number_of_presymptomatic_students = number_of_presymptomatic_students + 1
            elif student.HEALTH_STATUS == 'Mildly_symptomatic':
                number_of_mildly_symptomatic_students = number_of_mildly_symptomatic_students + 1
            elif student.HEALTH_STATUS == 'Symptomatic':
                number_of_symptomatic_students = number_of_symptomatic_students + 1
            elif student.HEALTH_STATUS == 'Immune':
                number_of_immune_students = number_of_immune_students + 1
            elif student.HEALTH_STATUS == 'Deceased':
                number_of_deseased_students = number_of_deseased_students + 1
                
        number_of_infected_students = number_of_asymptomatic_students + number_of_presymptomatic_students + number_of_mildly_symptomatic_students + number_of_symptomatic_students

        new_row = {'time': time, 'Healthy' : number_of_healthy_students, 'Exposed' : number_of_exposed_students, 'Asymptomatic' : number_of_asymptomatic_students, 'Pre_symptomatic' : number_of_presymptomatic_students,  'Mildly_symptomatic' : number_of_mildly_symptomatic_students,  'Symptomatic' : number_of_symptomatic_students,  'Immune' : number_of_immune_students,  'Deceased' : number_of_deseased_students, 'Teacher': teacher_status}
        return(new_row)  

In [7]:
def traditional_Wells_Riley_risk(classroom_grid, rows, columns, d, classroom_height, ERq, IVRR, n0, T):
    I = 0
    for row in classroom_grid:
        for agent in row:
            if (agent.infection_status == 1) & (agent.class_presence == 1):#students who are infected
                I = I + 1

    V = (rows + 1)*columns*d*d*classroom_height #don't forget the teacher!

    def f(t):
        R = (ERq * I)/(IVRR*V) + (n0 + ERq * I/IVRR)*np.exp(-IVRR*t)/V
        return R

    results = []
    t_range = np.arange(0, T+1, 1) #in mins
    for ttt in list(t_range):
        results.append(1 - np.exp(-IR*(integrate.quad(f,0,ttt/60)[0])))       
        
    return results

## For further information please contact rzz5164@psu.edu